In [1]:
import spacy
import numpy as np
import itertools

C:\Users\melis\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
nlp_ner = spacy.load('model-best')

OSError: [E050] Can't find model 'model-best'. It doesn't seem to be a Python package or a valid path to a data directory.

In [30]:
doc = nlp_ner("3 öğretmen 3 öğrenci Kasım 23 salı gidip 30 Ekim dönen İstanbul treni için yer sorgulamak istiyorum")

colors = {"DURAK": "#F67DE3", "YOLCU": "#7DF6D9", "SAYI":"#a6e22d", "AY":"#FF5733", "GÜN":"#2D14FF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [31]:
class Extract():
    """
    Extract information from the received request and return a url as a response
    """
    def __init__(self):
        
        self.durak_list = ['Ankara', 'İstanbul', 'Eskişehir', 'İzmir', 'Kars', 'Konya', 'Malatya', 'Adana', 'Polatlı', 'Söğütlüçeşme', 'Eryaman', 'Bakırköy', 'Bozüyük', 'Sakarya']
        self.ay_list = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
        self.gün_list = ['pazartesi', 'salı', 'çarşamba', 'perşembe', 'cuma', 'cumartesi', 'pazar']
        self.birey_list = ['ben', 'bana', 'benim adıma', 'kendim', 'kendime']
        self.ö_erteleme_list = ['güne', 'gün sonraya', 'gün sonrasına', 'gün ilerisine', 'ay', 'hafta', 'ayın', 'haftasına']
        self.cinsiyet_list = ['erkek', 'kadın', 'kız', 'hanımefendi', 'beyefendi']
        self.erteleme_list = ['haftaya', 'günübirlik', 'haftaya yarın', 'ertesi', 'ertesi gün', 'ertesi güne', 'yarın', 'yarına', 'yarın için', 'bugün', 'bugüne', 'haftaya bugün', 'sabaha', 'akşama', 'akşam için', 'sabah için', 'öğleye', 'öğlene', 'öğleden sonraya', 'öğleden sonrasına', 'hafta içi', 'hafta sonu']
        self.yolcu_list = ['yolcu', 'engelli', 'arkadaş', 'askeri personel', 'sakat', 'gazi', 'kişilik', 'kişiye', 'şehit yakını', 'gazi yakını', 'hamile', 'çocuklu', 'arkadaşım', 'veli', 'yetişkin', 'çocuğum', 'yaşlı', 'kişi', 'çocuk', 'hayvan', 'öğrenci', 'genç', 'öğretmen', 'öğretim görevlisi', 'basın', 'muhabir', 'asker', 'personel', 'çalışan', '60 yaş', '65 yaş']
        self.sayı_list = list(np.arange(1,32).astype(str))
        
        self.response_content = {}

        self.response_content['Kalkış'] = ""
        self.response_content['Varış'] = ""
        
        self.response_content['KalkışTarihi'] = ""
        self.response_content['VarışTarihi'] = ""
        
        self.response_content['YolcuTipi'] = []
        self.response_content['YolcuSayısı'] = []
        
        self.response_content['url'] = ""
        
        self.entity_list = {'DURAK':self.durak_list,
                            'AY':self.ay_list,
                            'YOLCU':self.yolcu_list,
                            'GÜN':self.gün_list,
                            'BİREY':self.birey_list,
                            'ERTELEME':self.erteleme_list,
                            'Ö-ERTELEME':self.ö_erteleme_list,
                            'CİNSİYET':self.cinsiyet_list,
                            'SAYI':self.sayı_list}
        
    def extract_entities(self, doc):
        
        duraklar = []
        aylar = []
        yolcular = []
        günler = []
        bireyler = []
        ertelemeler = []
        ö_ertelemeler = []
        cinsiyetler = []
        yolcu_sayıları = []
        tarihler = []
        
        last_ent = ""
        reverse_entities = doc.ents[::-1]
        for ent in reverse_entities:
            start = ent.start_char
            end = ent.end_char
            index = (start, end)
            label = ent.label_
            ent_list = self.entity_list[label]
            if str(ent) in ent_list:
                if label == 'DURAK': duraklar.append(str(ent)) 

                if label == 'AY': aylar.append(str(ent)) 

                if label == 'YOLCU': yolcular.append(str(ent)) 

                if label == 'GÜN': günler.append(str(ent)) 

                if label == 'BİREY': bireyler.append(str(ent)) 
                
                if label == 'ERTELEME': ertelemeler.append(str(ent)) 

                if label == 'Ö-ERTELEME': ö_ertelemeler.append(str(ent)) 
                
                if label == 'CİNSİYET': cinsiyetler.append(str(ent)) 
                
                if label == 'SAYI': 
                    if last_ent == 'YOLCU':
                        yolcu_sayıları.append(str(ent))
                    else:
                        tarihler.append(str(ent))
                
                    
            last_ent = label
            
        entities = [duraklar, aylar, yolcular, günler, bireyler, ertelemeler, ö_ertelemeler, cinsiyetler, tarihler, yolcu_sayıları]
        return entities
                
    def extract_stations(self, entities):
        
        duraklar = entities[0]
        
        if len(duraklar) == 2:
            varış, kalkış = duraklar
            self.response_content['Kalkış'] = kalkış
            self.response_content['Varış'] = varış
            
        if len(duraklar) == 1: 
            
            # Assign default_location as the departure location that is retrieved from GPS
            kalkış, varış = 'default_location', duraklar[0]

            self.response_content['Kalkış'] = kalkış
            self.response_content['Varış'] = varış
            
            
    def extract_dates(self, entities):
        
        tarihler = entities[8][::-1]
        aylar = entities[1][::-1]
        günler = entities[3][::-1]
        tam_tarih = list(itertools.zip_longest(aylar, tarihler, fillvalue=''))
        
        if len(günler) == 0:
    
            if len(aylar) == 2 and len(tarihler) == 2:
                self.response_content['KalkışTarihi'] = f"{tam_tarih[0][1]} {tam_tarih[0][0]}"
                self.response_content['VarışTarihi'] = f"{tam_tarih[1][1]} {tam_tarih[1][0]}"
                
            if len(aylar) == 1 and len(tarihler) == 1:
                self.response_content['KalkışTarihi'] = 'tomorrow'
                self.response_content['VarışTarihi'] = f"{tam_tarih[0][1]} {tam_tarih[0][0]}"
                
         
        if len(günler) == 1:
            
            if len(aylar) == 1:
                self.response_content['KalkışTarihi'] = günler[0]
                self.response_content['VarışTarihi'] = f"{tam_tarih[0][1]} {tam_tarih[0][0]}"
                
            if len(aylar) == 2 and len(tarihler) == 2:
                self.response_content['KalkışTarihi'] = f"{tam_tarih[0][1]} {tam_tarih[0][0]}"
                self.response_content['VarışTarihi'] = f"{tam_tarih[1][1]} {tam_tarih[1][0]}"
                
            elif len(aylar) == 1 and len(tarihler) == 1:
                self.response_content['KalkışTarihi'] = 'tomorrow'
                self.response_content['VarışTarihi'] = f"{tam_tarih[0][1]} {tam_tarih[0][0]}"
                
            else:
                self.response_content['KalkışTarihi'] = günler[0]
        
            
        if len(günler) == 2:
            
            self.response_content['KalkışTarihi'] = günler[0]
            self.response_content['KalkışTarihi'] = günler[1] 
    
        
    def extract_passengers(self, entities):
        
        yolcular = entities[2][::-1]
        yolcu_sayıları = entities[9][::-1]
        
        self.response_content['YolcuTipi'] = yolcular
        self.response_content['YolcuSayısı'] = yolcu_sayıları
            
    def extract(self, doc):
        
        entities = self.extract_entities(doc)
        self.extract_stations(entities)
        self.extract_dates(entities)
        self.extract_passengers(entities)

In [32]:
extract = Extract()

In [33]:
extract.extract(doc)
extract.response_content

{'Kalkış': 'default_location',
 'Varış': 'İstanbul',
 'KalkışTarihi': '23 Kasım',
 'VarışTarihi': '30 Ekim',
 'YolcuTipi': ['öğretmen', 'öğrenci'],
 'YolcuSayısı': ['3', '3'],
 'url': ''}